# YOLOv8-seg LGG MRI Tumor Segmentation


In [ ]:
# CELL 1: Clean install – fixes ALL errors (libGL, CUDA, old ultralytics)
!pip uninstall -y ultralytics torch torchvision torchaudio opencv-python opencv-python-headless -q 2>/dev/null

# Install PyTorch + CUDA 11.8 (perfect for Codespaces)
!pip install -q --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install latest YOLOv8 + kaggle + headless OpenCV (no GUI = no libGL error)
!pip install -q --no-cache-dir ultralytics kaggle tqdm
!pip install -q --no-cache-dir opencv-python-headless==4.10.0.84

import torch
import ultralytics
import cv2, os, random, numpy as np
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO
from IPython.display import clear_output, display
from IPython.display import FileLink

print("Installation completed successfully!")
print(f"Ultralytics YOLOv8 version: {ultralytics.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# CELL 2: Upload kaggle.json
print("Please DRAG & DROP your kaggle.json file into the left panel")
print("Then run this cell again")
clear_output(wait=True)

if not Path("kaggle.json").exists():
    print("kaggle.json not found! Upload it first.")
else:
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configured successfully!")

In [ ]:
# CELL 3: Download & extract dataset
!rm -rf lgg-mri-segmentation lgg-mri-segmentation.zip 2>/dev/null
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation --unzip

patients = [p for p in Path("lgg-mri-segmentation").iterdir() if p.is_dir()]
print(f"Dataset downloaded → {len(patients)} patient folders found")
Example: {patients[:3]}")

In [ ]:
# CELL 4: Convert to YOLOv8 segmentation format
!rm -rf yolo_dataset 2>/dev/null

raw = Path("lgg-mri-segmentation")
yolo = Path("yolo_dataset")
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    (yolo / folder).mkdir(parents=True, exist_ok=True)

pairs = []
for patient in raw.iterdir():
    if patient.is_dir():
        for img in patient.glob("*[0-9].tif"):
            mask = patient / f"{img.stem}_mask.tif"
            if mask.exists():
                pairs.append((img, mask))

random.seed(42)
random.shuffle(pairs)
split = int(0.8 * len(pairs))

def convert_to_yolo(img_path, mask_path, folder):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), 0)
    h, w = img.shape[:2]
    name = f"{img_path.parent.name}_{img_path.name}"
    
    cv2.imwrite(str(yolo/f"images/{folder}/{name}"), img)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lines = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 30:
            c = cnt.flatten().astype(float)
            c[0::2] /= w
            c[1::2] /= h
            lines.append("0 " + " ".join(map(str, c)))
    
    if lines:
        with open(yolo/f"labels/{folder}/{name.replace('.tif','.txt')}", "w") as f:
            f.write("\n".join(lines))

print("Converting training set...")
for p in tqdm(pairs[:split]): convert_to_yolo(p[0], p[1], "train")
print("Converting validation set...")
for p in tqdm(pairs[split:]): convert_to_yolo(p[0], p[1], "val")

# Create data.yaml
with open("data.yaml", "w") as f:
    f.write("""path: yolo_dataset
train: images/train
val: images/val
nc: 1
names: ['tumor']
""")

print(f"YOLOv8 dataset ready! → {len(pairs[:split])} train, {len(pairs[split:])} val images")

In [ ]:
# CELL 5: Train YOLOv8-seg on GPU (25–35 minutes)
model = YOLO("yolov8n-seg.pt")  # nano = fast & good for demo

results = model.train(
    data="data.yaml",
    epochs=80,
    imgsz=256,
    batch=32,
    device=0,
    project="runs",
    name="yolov8_lgg_final",
    exist_ok=True,
    patience=15,
    amp=True,
    workers=4,
    seed=42,
    plots=True
)

print("YOLOv8 training completed successfully!")

In [ ]:
# CELL 6: Create beautiful 30-frame demo video
best_model = YOLO("runs/yolov8_lgg_final/weights/best.pt")

val_images = sorted(list(Path("yolo_dataset/images/val").glob("*.tif")))[:30]
frames = []

print("Generating demo video...")
for img_path in tqdm(val_images):
    img = cv2.imread(str(img_path))
    results = best_model(img_path, conf=0.25, verbose=False)[0]
    
    overlay = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if results.masks is not None:
        for mask in results.masks.data.cpu():
            m = cv2.resize((mask.numpy() > 0.5).astype(np.uint8), (img.shape[1], img.shape[0]))
            overlay = np.where(m[...,None], overlay*0.6 + np.array([0, 200, 255])*0.4, overlay)
    
    cv2.putText(overlay, "YOLOv8-seg | LGG MRI Tumor Segmentation", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 255), 3)
    cv2.putText(overlay, "Omid Sakaki", (30, overlay.shape[0]-30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (200, 255, 255), 2)
    
    frames.append(overlay)

# Save video
h, w = frames[0].shape[:2]
out = cv2.VideoWriter("YOLOv8_LGG_MRI_Demo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 6, (w, h))
for frame in frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
out.release()

display(FileLink("YOLOv8_LGG_MRI_Demo.mp4"))
print("FINISHED! Click the link above to download your professional demo video!")